# Load libraries and functions

In [476]:
with open('libraries.py') as f:
    code = f.read()
exec(code)

with open('functions.py') as f:
    code = f.read()
exec(code)

In [477]:
# determine user
user = getpass.getuser()
if user == 'peymansh':
    main_folder_path = '/Users/peymansh/Dropbox (MIT)/Research/AI and Occupations/ai-exposure'
    data_path = f'{main_folder_path}/output'

## Main Code

In [478]:
# Pick occupation and initialize variables
occupation = 'travelAgents'
occupation = 'insuranceUnderwriters'
occupation = 'pileDriverOperators'
occupation = 'dredgeOperators'
occupation = 'gradersAndSorters'
occupation = 'reinforcingIron'
occupation = 'insuranceAppraisers'
occupation = 'floorSanders'
occupation = 'dataEntryKeyer'
# occupation = 'athletesAndSportsCompetitors'
# # occupation = 'shampooers'

GPT_input_occupation, plot_title_occupation, occupation_code, occupation_folder = pick_occupation(occupation)

In [479]:
# Load the data
onet = pd.read_csv(f'{data_path}/data/onet_occupations_yearly.csv')
onet = onet.sort_values(by=['year', 'occ_code', 'occ_title', 'task_id'])
onet = onet[onet['year'] == 2023].reset_index(drop=True)

# Get list of tasks
my_df = onet[(onet.occ_code == f'{occupation_code}') & (onet.year == 2023)]
tasks = my_df['task'].unique().tolist()
tasks

['Read source documents such as canceled checks, sales reports, or bills, and enter data in specific data fields or onto tapes or disks for subsequent entry, using keyboards or scanners.',
 'Compile, sort, and verify the accuracy of data before it is entered.',
 'Compare data with source documents, or re-enter data in verification format to detect errors.',
 'Store completed documents in appropriate locations.',
 'Locate and correct data entry errors, or report them to supervisors.',
 'Maintain logs of activities and completed work.',
 'Select materials needed to complete work assignments.',
 'Load machines with required input or output media, such as paper, cards, disks, tape, or Braille media.',
 'Resolve garbled or indecipherable messages, using cryptographic procedures and equipment.']

<br>

<br>

# 5) GPT Triangles/Conditioning Method

#### Approach: Use either First-Last Task method or Partitioned Tasks Method in creating original GPT DAG. Next use the "Triangles" or "Conditioning" method for narrowing down set of edges.

In [480]:
version = 'v0/'
version = 'v1/'
version = ''

In [481]:
input_version = 'firstLastTask'
#input_version = 'partitioned'

### Set input-output file names for different versions of the run

In [482]:
if input_version == 'firstLastTask':
    input_filename = f'{occupation_folder}/{version}{occupation}_firstLastTaskGPT_DAG_df.csv'
    output_filename = f'{occupation_folder}/{version}{occupation}_conditionedGPT_fromFirstLastTask_DAG_df.csv'

if input_version == 'partitioned':
    input_filename = f'{occupation_folder}/{version}{occupation}_partitionedGPT_DAG_df.csv'
    output_filename = f'{occupation_folder}/{version}{occupation}_conditionedGPT_fromPartitioned_DAG_df.csv'

### Set up questions and options

In [483]:
# if version == 'v0/':
#     triangle_check_question_text = dedent("""\
#                                           Consider {{ occupation }} as an occupation. 
#                                           And consider these three tasks: 
#                                           A) {{ task_A }} 
#                                           B) {{ task_B }}
#                                           C) {{ task_C }} 
#                                           What are the prerequisites of doing task C?
#                                           Avoid using words like "task A", "task B", or "task C" in the answer.
#                                           Explain the reasoning behind your answer in a couple of sentences.
#                                           """)
#     triangle_check_question_options = {'keep_AC': "C can be done after A without having to do B",
#                                        'drop_AC': "C can only be done after B",
#                                        'sanity_check': "These are not part of the same task sequence"
#                                        }
    
#     AC_DC_question_question_text = dedent("""\
#                                           Consider {{ occupation }} as an occupation.
#                                           And consider these tasks:
#                                           A) {{ task_A }}
#                                           B) {{ task_B }}
#                                           C) {{ task_C }}
#                                           D) {{ task_D }}         
#                                           As part of the steps leading up to completion of this job '{{ task_B }}' is done after '{{ task_A }}' but before '{{ task_C }}'.
#                                           Furthermore, '{{ task_A }}' is done after '{{ task_D }}' but before '{{ task_C }}'.
#                                           Given this structure, determine if task A and task D are direct prerequisites of doing task C?
#                                           Avoid using words like "task A", "task B", or "task C" in the answer.
#                                           Explain the reasoning behind your answer in a couple of sentences.
#                                           """)
#     AC_DC_question_options = {'drop AC, drop DC': "C can be done after B only after both A and D have been done earlier", 
#                               'keep AC, drop DC': "C can be done after A only after having done D first, but without having to do B",
#                               'drop AC, keep DC': "C can be done immediately after D without having to do A or B, but it cannot be done after A without having done D or B first",
#                               'keep AC, keep DC': "C can be done immediately after D without having to do A or B, and it can also be done after A without having done D or B",
#                               'sanity check': "These are not part of the same task sequence"
#                               }
    


if version == 'v0/':
    triangle_check_question_text = dedent("""\
                                          Consider {{ occupation }} as an occupation. 
                                          And consider these three tasks: 
                                          A) {{ task_A }} 
                                          B) {{ task_B }}
                                          C) {{ task_C }} 
                                          What are the prerequisites of doing task C?
                                          Avoid using words like "task A", "task B", or "task C" in the answer.
                                          Explain the reasoning behind your answer in a couple of sentences.
                                          """)
    triangle_check_question_options = {'keep AC': "Given that B is done, it is also necessary to do A in order to do C",
                                       'drop AC': "Given that B is done, doing A is not needed in order to do C",
                                       'sanity check': "These are not part of the same task sequence"
                                       }
    
    AC_DC_question_question_text = dedent("""\
                                          Consider {{ occupation }} as an occupation.
                                          And consider these tasks:
                                          A) {{ task_A }}
                                          B) {{ task_B }}
                                          C) {{ task_C }}
                                          D) {{ task_D }}         
                                          As part of the steps leading up to completion of this job {{ task_C }} must be done. 
                                          We know that tasks A, B, and D are inputs to task C.
                                          Moreover, task D is an input to task A while task A is an input to task B.
                                          Given that task B is done, do tasks A and D need to be done before doing task C?
                                          Avoid using words like "task A", "task B", or "task C" in the answer.
                                          Explain the reasoning behind your answer in a couple of sentences.
                                          """)
    AC_DC_question_options = {'drop AC, drop DC': "Given that B is done, doing neither A nor D is needed in order to do C",
                              'keep AC, drop DC': "Given that B is done, doing A is needed for doing C but doing D is not",
                              'drop AC, keep DC': "Given that B is done, doing D is needed for doing C but doing A is not",
                              'keep AC, keep DC': "Given that B is done, doing both A and D are needed for doing C",
                              'sanity check': "These are not part of the same task sequence"
                              }
    

In [484]:
if version == 'v1/':
    triangle_check_question_text = dedent("""\
                                          Consider {{ occupation }} as an occupation. 
                                          And consider these three tasks: 
                                          A) {{ task_A }} 
                                          B) {{ task_B }}
                                          C) {{ task_C }} 
                                          Imagine there are two workers, one is working on task A and the other is working on task B.
                                          Suppose a third worker wants to start working on task C.
                                          Does the third worker need to wait for the workers working on tasks A and B to finish before starting on task C?
                                          Or can the third worker start working on task C right after worker B finishes without needing output of worker A?
                                          Avoid using words like "task A", "task B", or "task C" in the answer.
                                          Explain the reasoning behind your answer in a couple of sentences.
                                          """)
    triangle_check_question_options = {'keep AC': "Third worker can only start C after both the first and second workers have finished tasks A and B",
                                       'drop AC': "Third worker can start C after the second worker finishes B without having to wait for the first worker to finish A",
                                       'sanity check': "These are not part of the same task sequence"
                                       }
    
    AC_DC_question_question_text = dedent("""\
                                          Consider {{ occupation }} as an occupation.
                                          And consider these tasks:
                                          A) {{ task_A }}
                                          B) {{ task_B }}
                                          C) {{ task_C }}
                                          D) {{ task_D }}         
                                          As part of the steps leading up to completion of this job {{ task_C }} must be done.
                                          We know that tasks A, B, and D are inputs to task C.Moreover, task D is an input to task A while task A is an input to task B.
                                          Imagine there are three workers, one is working on task A, one is working on task B, and the other is working on task D.
                                          Suppose a fourth worker wants to start working on task C.
                                          Does the fourth worker need to wait for the workers working on tasks A, B, and D to finish before starting on task C?
                                          Or can the third worker start working on task C after workers B and D finish without needing output of worker A?
                                          How about after workers B and A finish without needing output of worker D?
                                          Avoid using words like "task A", "task B", or "task C" in the answer.
                                          Explain the reasoning behind your answer in a couple of sentences.
                                          """)
    AC_DC_question_options = {'drop AC, drop DC': "Given that worker B is finished, output of neither worker A nor worker D is needed for worker C",
                              'keep AC, drop DC': "Given that worker B is finished, output of worker A is needed for worker C but output of worker D is not",
                              'drop AC, keep DC': "Given that worker B is finished, output of worker D is needed for worker C but output of worker A is not",
                              'keep AC, keep DC': "Given that worker B is finished, output of both workers A and D are needed for worker C",
                              'sanity check': "These are not part of the same task sequence"
                              }

In [485]:
if (version != 'v1/') & (version != 'v0/'): # same as 'v1/'
    triangle_check_question_text = dedent("""\
                                          Consider {{ occupation }} as an occupation. 
                                          And consider these three tasks: 
                                          A) {{ task_A }} 
                                          B) {{ task_B }}
                                          C) {{ task_C }} 
                                          Imagine there are two workers, one is working on task A and the other is working on task B.
                                          Suppose a third worker wants to start working on task C.
                                          Does the third worker need to wait for the workers working on tasks A and B to finish before starting on task C?
                                          Or can the third worker start working on task C right after worker B finishes without needing output of worker A?
                                          Avoid using words like "task A", "task B", or "task C" in the answer.
                                          Explain the reasoning behind your answer in a couple of sentences.
                                          """)
    triangle_check_question_options = {'keep AC': "Third worker can only start C after both the first and second workers have finished tasks A and B",
                                       'drop AC': "Third worker can start C after the second worker finishes B without having to wait for the first worker to finish A",
                                       'sanity check': "These are not part of the same task sequence"
                                       }
    
    AC_DC_question_question_text = dedent("""\
                                          Consider {{ occupation }} as an occupation.
                                          And consider these tasks:
                                          A) {{ task_A }}
                                          B) {{ task_B }}
                                          C) {{ task_C }}
                                          D) {{ task_D }}         
                                          As part of the steps leading up to completion of this job {{ task_C }} must be done.
                                          We know that tasks A, B, and D are inputs to task C.Moreover, task D is an input to task A while task A is an input to task B.
                                          Imagine there are three workers, one is working on task A, one is working on task B, and the other is working on task D.
                                          Suppose a fourth worker wants to start working on task C.
                                          Does the fourth worker need to wait for the workers working on tasks A, B, and D to finish before starting on task C?
                                          Or can the third worker start working on task C after workers B and D finish without needing output of worker A?
                                          How about after workers B and A finish without needing output of worker D?
                                          Avoid using words like "task A", "task B", or "task C" in the answer.
                                          Explain the reasoning behind your answer in a couple of sentences.
                                          """)
    AC_DC_question_options = {'drop AC, drop DC': "Given that worker B is finished, output of neither worker A nor worker D is needed for worker C",
                              'keep AC, drop DC': "Given that worker B is finished, output of worker A is needed for worker C but output of worker D is not",
                              'drop AC, keep DC': "Given that worker B is finished, output of worker D is needed for worker C but output of worker A is not",
                              'keep AC, keep DC': "Given that worker B is finished, output of both workers A and D are needed for worker C",
                              'sanity check': "These are not part of the same task sequence"
                              }

In [486]:
triangle_check_question_options_list = list(triangle_check_question_options.values())
AC_DC_question_options_list = list(AC_DC_question_options.values())

In [487]:
def triangle_check(occupation, tasks, triangles_list, question_text, question_options_list):
    triangles = np.array(triangles_list)
    task_A_list = triangles[:, 0]
    task_B_list = triangles[:, 1]
    task_C_list = triangles[:, 2]
    scenarios = [Scenario({"occupation": occupation, "task_A": tasks[task_A], "task_B": tasks[task_B], "task_C": tasks[task_C]}) 
        for task_A, task_B, task_C in zip(task_A_list, task_B_list, task_C_list)]

    q = QuestionMultipleChoice(
        question_name = "ordering",
        question_text = question_text,
        question_options = question_options_list
    )
    results = q.by(m4).by(scenarios).run(progress_bar = True)
    return results



q_AC_DC = QuestionMultipleChoice(
    question_name = "AC_DC",
    question_text = AC_DC_question_question_text,
    question_options = AC_DC_question_options_list
    )

<br> 

### Step 1:

#### Find all "triangles", defined as cases with:
##### A --> B --> C
##### A --> C

In [488]:
# Read output of one step GPT DAG
GPT_AM_df = pd.read_csv(input_filename)

# Convert GPT AM data frame to adjacency matrix
GPT_AM = pd.DataFrame(0, index=tasks, columns=tasks)
for index, row in GPT_AM_df.iterrows():
    GPT_AM.at[row['source'], row['target']] = 1

In [489]:
def find_triangles(matrix):
    # Ensure matrix is a numpy array
    if not isinstance(matrix, np.ndarray):
        matrix = matrix.to_numpy()
    
    # get length of matrix
    n = matrix.shape[0]

    # create list containing integers from 0 to n-1 for indexing
    numbers = list(range(n))

    # Find triangles
    triangles = []
    for x, y, z in itertools.permutations(numbers, 3):
        # get indices of destination nodes for outgoing edges of x
        out_edges_destination_x = np.where(matrix[x] == 1)[0]
        out_edges_destination_x = list(out_edges_destination_x)

        # check if x has outgoing edge to both y and z
        # if yes, check if y has outgoing edge to z
        if y in out_edges_destination_x and z in out_edges_destination_x:
            out_edges_destination_y = np.where(matrix[y] == 1)[0]
            out_edges_destination_y = list(out_edges_destination_y)
            
            # check if y has outgoing edge to z
            # if yes, we have a triangle
            if z in out_edges_destination_y:
                triangles.append([x, y, z])
    
    return triangles

# Find triangles
GPT_AM_triangles_list = find_triangles(GPT_AM)
print(f'Examples of triangles: {GPT_AM_triangles_list[:5]}')
print(f'Count of triangles: {len(GPT_AM_triangles_list)}')

Examples of triangles: [[0, 1, 2], [0, 1, 3], [0, 1, 4], [0, 1, 5], [0, 2, 3]]
Count of triangles: 25


In [490]:
# If there are no triangles found, export input as output (as conditioning method doesn't change anything)
if len(GPT_AM_triangles_list) == 0:
    GPT_AM_df.to_csv(output_filename, index=False)

### Step 2: 
#### Ask GPT whether conditional on having B --> C we need A --> C

In [491]:
results = triangle_check(GPT_input_occupation, tasks, GPT_AM_triangles_list, triangle_check_question_text, triangle_check_question_options_list)
#results.select("task_A", "task_B", "task_C", "ordering", "comment.ordering_comment").print()
GPT_trianglesCheck_output = results.select("task_A", "task_B", "task_C", "ordering", "comment.ordering_comment").to_pandas()
GPT_trianglesCheck_output = GPT_trianglesCheck_output.sort_values(by=['scenario.task_A', 'scenario.task_C', 'scenario.task_B']).reset_index(drop=True)

Output()

### In cases where A --> C is shared among multiple triangles, only delete when all triangles say delete

In [492]:
# Step 1: Find the count of triangles for each A --> C pair
GPT_trianglesCheck_output['AC_pair_triangles_count'] = GPT_trianglesCheck_output.groupby(['scenario.task_A', 'scenario.task_C'])['scenario.task_A'].transform('count')


# Step 2: Find if all triangles say delete
aux_df = GPT_trianglesCheck_output.groupby(['scenario.task_A', 'scenario.task_C'])['answer.ordering'].apply(lambda x: (x == triangle_check_question_options['drop AC']).mean()*100).reset_index()
aux_df.columns = ['scenario.task_A', 'scenario.task_C', 'fraction_triangles_say_delete']
aux_df = aux_df[aux_df['fraction_triangles_say_delete'] == 100]

# Merge aux_df with the original DataFrame to keep 'comment.ordering_comment'
edges_to_remove = pd.merge(aux_df, GPT_trianglesCheck_output, on=['scenario.task_A', 'scenario.task_C'])
edges_to_remove['comment.ordering_comment'] = (
    edges_to_remove['comment.ordering_comment'] 
    + '\n\n(Source):\n' + edges_to_remove['scenario.task_A'] 
    + '\n(Conditioned on):\n' + edges_to_remove['scenario.task_B'] 
    + '\n(Target):\n' + edges_to_remove['scenario.task_C']
)
edges_to_remove = edges_to_remove[['scenario.task_A', 'scenario.task_C', 'comment.ordering_comment']]

# Step 3: Delete the rows where all triangles say delete
modified_GPT_trianglesCheck = pd.merge(GPT_trianglesCheck_output, edges_to_remove[['scenario.task_A', 'scenario.task_C']], 
                                       how='left', 
                                       on=['scenario.task_A', 'scenario.task_C'], 
                                       indicator=True)
modified_GPT_trianglesCheck = modified_GPT_trianglesCheck[modified_GPT_trianglesCheck['_merge'] == 'left_only'].drop(columns=['_merge', 'AC_pair_triangles_count'])
modified_GPT_trianglesCheck = modified_GPT_trianglesCheck.reset_index(drop=True)
edges_to_remove

,scenario.task_A,scenario.task_C,comment.ordering_comment
0,"Compare data with source documents, or re-ente...",Maintain logs of activities and completed work.,The third worker can start maintaining logs of...
1,"Compare data with source documents, or re-ente...",Maintain logs of activities and completed work.,The third worker can start maintaining logs of...
2,Load machines with required input or output me...,"Locate and correct data entry errors, or repor...",The third worker can start locating and correc...
3,"Locate and correct data entry errors, or repor...",Maintain logs of activities and completed work.,The third worker can start maintaining logs of...
4,Select materials needed to complete work assig...,"Locate and correct data entry errors, or repor...",The third worker can start correcting data ent...
5,Select materials needed to complete work assig...,"Locate and correct data entry errors, or repor...",The third worker can start locating and correc...
6,Select materials needed to complete work assig...,"Read source documents such as canceled checks,...",The third worker can start entering data once ...


#### Create a variable saying how many times each node appears as which node in a triangle
##### Purpose: find quanrangles

In [493]:
# Initialize an empty DataFrame with unique values as columns and original columns as rows
aux_df = pd.DataFrame(0, index=['scenario.task_A', 'scenario.task_B', 'scenario.task_C'], columns=tasks)

# Fill the new DataFrame with counts
for col in modified_GPT_trianglesCheck[['scenario.task_A', 'scenario.task_B', 'scenario.task_C']].columns:
    value_counts = modified_GPT_trianglesCheck[col].value_counts()
    aux_df.loc[col, value_counts.index] = value_counts.values
aux_df = aux_df.T

# Keep tasks which are sometimes node A of a triangle and sometimes node B of a triangle
#aux_df = aux_df[(aux_df > 0).all(axis=1)]
print('Nodes stats as nodes A, B, C of a triangle:')
aux_df

# get list of pivotal tasks
#pivotal_tasks = aux_df.index.tolist()

Nodes stats as nodes A, B, C of a triangle:


,scenario.task_A,scenario.task_B,scenario.task_C
"Read source documents such as canceled checks, sales reports, or bills, and enter data in specific data fields or onto tapes or disks for subsequent entry, using keyboards or scanners.",10,0,0
"Compile, sort, and verify the accuracy of data before it is entered.",7,4,0
"Compare data with source documents, or re-enter data in verification format to detect errors.",1,6,1
Store completed documents in appropriate locations.,0,2,6
"Locate and correct data entry errors, or report them to supervisors.",0,5,4
Maintain logs of activities and completed work.,0,0,7
Select materials needed to complete work assignments.,0,1,0
"Load machines with required input or output media, such as paper, cards, disks, tape, or Braille media.",0,0,0
"Resolve garbled or indecipherable messages, using cryptographic procedures and equipment.",0,0,0


### Step 3: 
##### In cases where 
A --> B --> C and D --> A --> C 
##### the situation is different from when 
A --> B --> C and A --> D --> C
##### In such cases, edges A --> C and D --> C must be considered simultaneously as triangles are not totally "independent". 
#### So we look for "quadrangles"

In [494]:
# Iterate over the list of tuples and subset the DataFrame
quadrangles_tasks = []
for A, B, C, D in itertools.permutations(tasks, 4):
    # Initialize an empty list to collect the indices of desired rows
    quadrangle_indices = []

    # Find rows where triangle nodes are A, B, C
    condition1 = (modified_GPT_trianglesCheck['scenario.task_A'] == A) & (modified_GPT_trianglesCheck['scenario.task_B'] == B) & (modified_GPT_trianglesCheck['scenario.task_C'] == C)
    rows1 = modified_GPT_trianglesCheck[condition1]
    
    # Find rows where triangle nodes are D, A, C
    condition2 = (modified_GPT_trianglesCheck['scenario.task_A'] == D) & (modified_GPT_trianglesCheck['scenario.task_B'] == A) & (modified_GPT_trianglesCheck['scenario.task_C'] == C)
    rows2 = modified_GPT_trianglesCheck[condition2]
    
    # If both conditions are met, add the indices to the list
    if not rows1.empty and not rows2.empty:
        quadrangles_tasks.append((A, B, C, D))    
print(f'Number of quadrilaterals: {len(quadrangles_tasks)}')

Number of quadrilaterals: 9


In [495]:
quadrangles_df = pd.DataFrame()
if len(quadrangles_tasks) > 0:
    scenarios = [Scenario({"occupation": GPT_input_occupation, "tasks": tasks,
                "task_A": A, "task_B": B, "task_C": C, "task_D": D})
                for A, B, C, D in quadrangles_tasks]
    results_AC_DC = q_AC_DC.by(m4).by(scenarios).run()
    #results_AC_DC.select(['answer.AC_DC', 'scenario.task_A', 'scenario.task_B', 'scenario.task_C', 'scenario.task_D', 'comment.AC_DC_comment']).print()
    quadrangles_df = results_AC_DC.select(['answer.AC_DC', 'scenario.task_A', 'scenario.task_B', 'scenario.task_C', 'scenario.task_D', 'comment.AC_DC_comment']).to_pandas()

    # decide whether to keep or drop AC and DC
    quadrangles_df['keep_AC'] = quadrangles_df['answer.AC_DC'].apply(lambda x: x in [AC_DC_question_options['keep AC, keep DC'], AC_DC_question_options['keep AC, drop DC']])
    quadrangles_df['keep_DC'] = quadrangles_df['answer.AC_DC'].apply(lambda x: x in [AC_DC_question_options['keep AC, keep DC'], AC_DC_question_options['drop AC, keep DC']])

    # Add node info to comments
    quadrangles_df['comment.AC_DC_comment'] = (
        quadrangles_df['comment.AC_DC_comment'] 
        + '\n\n(Source):\n' + quadrangles_df['scenario.task_A'] 
        + '\n(Conditioned on):\n' + quadrangles_df['scenario.task_B'] 
        + '\n(Target):\n' + quadrangles_df['scenario.task_C']
        + '\n(Other task -- Task D):\n' + quadrangles_df['scenario.task_D']
    )
quadrangles_df.head()

,answer.AC_DC,comment.AC_DC_comment,scenario.task_A,scenario.task_B,scenario.task_C,scenario.task_D,keep_AC,keep_DC
0,"Given that worker B is finished, output of bot...",The fourth worker needs the outputs of both th...,"Compile, sort, and verify the accuracy of data...","Compare data with source documents, or re-ente...",Store completed documents in appropriate locat...,"Read source documents such as canceled checks,...",True,True
1,"Given that worker B is finished, output of bot...",The fourth worker needs the outputs from both ...,"Compile, sort, and verify the accuracy of data...","Compare data with source documents, or re-ente...","Locate and correct data entry errors, or repor...","Read source documents such as canceled checks,...",True,True
2,"Given that worker B is finished, output of bot...","The fourth worker, who is maintaining logs of ...","Compile, sort, and verify the accuracy of data...","Compare data with source documents, or re-ente...",Maintain logs of activities and completed work.,"Read source documents such as canceled checks,...",True,True
3,"Given that worker B is finished, output of bot...",The fourth worker needs the outputs from the w...,"Compile, sort, and verify the accuracy of data...",Store completed documents in appropriate locat...,Maintain logs of activities and completed work.,"Read source documents such as canceled checks,...",True,True
4,"Given that worker B is finished, output of bot...",The fourth worker needs to maintain logs of ac...,"Compile, sort, and verify the accuracy of data...","Locate and correct data entry errors, or repor...",Maintain logs of activities and completed work.,"Read source documents such as canceled checks,...",True,True


#### Drop extra AC and DC edges

In [496]:
ACDC_edges_to_remove = pd.DataFrame()
if len(quadrangles_tasks) > 0:
    # Step 1: Get list of unique edges found in all quadrangles
    pairs_AC = list(zip(quadrangles_df["scenario.task_A"], quadrangles_df["scenario.task_C"]))
    pairs_AC = [(task_A, task_C, 'AC') for (task_A, task_C) in pairs_AC]
    pairs_DC = list(zip(quadrangles_df["scenario.task_D"], quadrangles_df["scenario.task_C"]))
    pairs_DC = [(task_D, task_C, 'DC') for (task_D, task_C) in pairs_DC]
    all_pairs = pairs_AC + pairs_DC


    # Step 2: Get list of edges to keep
    aux_df = quadrangles_df[quadrangles_df['keep_AC']==True]
    pairs_AC_toKeep = list(zip(aux_df["scenario.task_A"], aux_df["scenario.task_C"]))
    pairs_AC_toKeep = [(task_A, task_C, 'AC') for (task_A, task_C) in pairs_AC_toKeep]

    aux_df = quadrangles_df[quadrangles_df['keep_DC']==True]
    pairs_DC_toKeep = list(zip(aux_df["scenario.task_D"], aux_df["scenario.task_C"]))
    pairs_DC_toKeep = [(task_D, task_C, 'DC') for (task_D, task_C) in pairs_DC_toKeep]

    pairs_toKeep = pairs_AC_toKeep + pairs_DC_toKeep


    # Step 3: Get list of edges to drop
    ACDC_edges_toDrop_list = [item for item in all_pairs if item not in pairs_toKeep]
    ACDC_edges_to_remove = pd.DataFrame(ACDC_edges_toDrop_list, columns=["scenario.task_A", "scenario.task_C", 'ID'])


    # Step 4: Match comments
    AC_indices = ACDC_edges_to_remove[ACDC_edges_to_remove['ID'] == 'AC'].index
    DC_indices = ACDC_edges_to_remove[ACDC_edges_to_remove['ID'] == 'DC'].index

    # Split ACDC_edges_to_remove into two DataFrames based on AC or DC edges
    aux_df_AC = ACDC_edges_to_remove.loc[AC_indices]#[['scenario.task_A', 'scenario.task_C']]
    aux_df_DC = ACDC_edges_to_remove.loc[DC_indices]#[['scenario.task_A', 'scenario.task_C']]

    # Merge comments depending on AC or DC edge
    merged_AC = pd.merge(aux_df_AC, quadrangles_df[['scenario.task_A', 'scenario.task_C', 'comment.AC_DC_comment']], 
                        on=['scenario.task_A', 'scenario.task_C'], 
                        how='left')
    merged_AC = merged_AC[['scenario.task_A', 'scenario.task_C', 'comment.AC_DC_comment']]
    merged_DC = pd.merge(aux_df_DC, quadrangles_df[['scenario.task_D', 'scenario.task_C', 'comment.AC_DC_comment']], 
                        left_on=['scenario.task_A', 'scenario.task_C'], 
                        right_on=['scenario.task_D', 'scenario.task_C'], 
                        how='left')
    merged_DC = merged_DC[['scenario.task_A', 'scenario.task_C', 'comment.AC_DC_comment']]

    # Concatenate merged DataFrames
    ACDC_edges_to_remove = pd.concat([merged_AC, merged_DC]).sort_index()
    ACDC_edges_to_remove = ACDC_edges_to_remove.drop_duplicates(subset=['scenario.task_A', 'scenario.task_C']).reset_index(drop=True)
    ACDC_edges_to_remove.columns = ['scenario.task_A', 'scenario.task_C', 'comment.ordering_comment'] # for consistency with previous edges_to_remove data frame


# Create a DataFrame of edges to be dropped from this analysis and earlier analyses
print(f'Number of AC-DC edges to remove: {len(ACDC_edges_to_remove)}')
print(f'Number of AC edges to remove: {len(edges_to_remove)}')
edges_to_remove = pd.concat([edges_to_remove, ACDC_edges_to_remove], ignore_index=True)
print(f'Total number of edges to remove: {len(edges_to_remove)}')

Number of AC-DC edges to remove: 0
Number of AC edges to remove: 7
Total number of edges to remove: 7


In [497]:
# # Remove redundant edges
# modified_GPT_AM_df = GPT_AM_df.copy()
# modified_GPT_AM_df = pd.merge(modified_GPT_AM_df, edges_to_remove, how='left', 
#                               left_on=['source', 'target'], right_on=['scenario.task_A', 'scenario.task_C'], 
#                               indicator=True)
# modified_GPT_AM_df = modified_GPT_AM_df[modified_GPT_AM_df['_merge'] == 'left_only'].drop(columns=['_merge', 'scenario.task_A', 'scenario.task_C'])
# modified_GPT_AM_df = modified_GPT_AM_df.reset_index(drop=True)


# Label edges to remove
def adjust_graph_label_for_removed_edges(edges_to_remove, main_df):
    output = main_df.copy()
    for _, row in edges_to_remove.iterrows():
        task_A, task_C, comment = row['scenario.task_A'], row['scenario.task_C'], row['comment.ordering_comment']
        match = (main_df['source'] == task_A) & (main_df['target'] == task_C)
        if match.any():
            index_to_update = main_df.index[match][0]  # Get the index where the pair matches
            output.at[index_to_update, 'comment'] = comment + 'TriangleRemovedFlag'
    return output
modified_GPT_AM_df = adjust_graph_label_for_removed_edges(edges_to_remove, GPT_AM_df)


# Save output
modified_GPT_AM_df.to_csv(output_filename, index=False)

<br>

<br>

<br>

<br>

<br>